---

Extrair noticias dos sites de verificação de fake news: 
<br> https://lupa.uol.com.br/jornalismo/categoria/verifica%C3%A7%C3%A3o 
<br> https://g1.globo.com/fato-ou-fake/
<br> https://apublica.org/tipo_nota/entrelinhas-do-poder/
<br> https://www.e-farsas.com/

---

In [ ]:

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

In [ ]:
'''
Cria uma instância do objeto que controla o início e parada do ChromeDriver
'''
s = Service('./chromedriver.exe')
'''
Define as opções do webdriver do Chrome
'''
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-notifications")
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-gpu')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36")


#### Carregando a página e scrollando para obter todos os links

In [ ]:
link = "https://lupa.uol.com.br/jornalismo/categoria/verifica%C3%A7%C3%A3o "
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=options) # abre o browser Chrome
driver.get(link)
sleep(5)

total_iterations = 100

for counter in tqdm(range(total_iterations), desc="Scrolling and Clicking"):
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.implicitly_wait(5)
        button_class = "sc-cwSeag jxHWSb"
        button_xpath = f"//button[@class='{button_class}']"
        button = WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.XPATH, button_xpath)))
        button.click()
        sleep(5)
    except Exception as e:
        print(f"An error occurred: {e}")
        break


#### Coletando as informações e os links das notícias

In [ ]:

elements = driver.find_elements(by=By.XPATH, value = "//div[@id='init']//div[@class='sc-dkrFOg cbDCWR']")
data = []
for element in elements:
    link = element.find_element(By.XPATH, "*//a[@class='sc-eDWCr hNENvd']").get_attribute('href')
    author = element.find_element(By.XPATH, ".//div[@class='sc-hLBbgP gREmsY']//span[@class='sc-eDvSVe hQCdpY']").text
    title = element.find_element(By.XPATH, ".//div[@class='sc-hLBbgP eRlKZk']//span").text
    headline = element.find_element(By.XPATH, ".//div[@class='sc-hLBbgP gQITEQ']//span/p").text
    data.append({'Author': author, 'Link': link, 'Title': title, 'Headline': headline})

In [ ]:
len(data)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
df.iloc[0]

In [ ]:
df.to_csv("links_lupa_uol.csv", index=False)

In [ ]:
df = pd.read_csv("links_lupa_uol.csv")

In [ ]:
df.insert(loc=0, column='RawText', value='')

In [ ]:
for i in tqdm(range(len(df)), desc="Access the page and extracting text"):
    
    link = df.iloc[i]['Link']

    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=options) 
    driver.get(link)
    sleep(3)
    
    elements = driver.find_elements(by=By.XPATH, value = "*//div[@id='root']")
    
    text_str = elements[0].text
    
    #Retira ruido antes de "Entrar" 
    text_str = text_str.split('Entrar')[1]
    
    #Retira ruido após "Leia Também"
    text_str = text_str.split('Leia também')[0]
    
    df.at[i, 'RawText'] = text_str
    
    driver.quit()

In [ ]:
df.head()

In [ ]:
df.to_csv("noticias_lupa.csv", index = False)